In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def data_loader():
    # load csv files:
    dataset_train = pd.read_csv('/Lab1/Lab5/train_data_stock.csv')
    dataset_val = pd.read_csv('/Lab1/Lab5/val_data_stock.csv')

    # reverse data so that they go from oldest to newest:
    dataset_train = dataset_train.iloc[::-1]
    dataset_val = dataset_val.iloc[::-1]

    # concatenate training and test datasets:
    dataset_total = pd.concat((dataset_train['Open'], dataset_val['Open']), axis=0) 

    # select the values from the “Open” column as the variables to be predicted:
    training_set = dataset_train.iloc[:, 1:2].values
    val_set = dataset_val.iloc[:, 1:2].values

    #normalize the data from 0 to 1
    sc = MinMaxScaler(feature_range=(0, 1))
    training_set_scaled = sc.fit_transform(training_set)


    # split training data into T time steps:
    X_train = []
    y_train = []
    T = 60

    for i in range(T, len(training_set)):
        X_train.append(training_set_scaled[i-T:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # normalize the validation set according to the normalization applied to the training set:
    inputs = dataset_total[len(dataset_total) - len(dataset_val) - 60:].values
    inputs = inputs.reshape(-1, 1)
    inputs = sc.transform(inputs)

    # split validation data into T time steps:
    X_val = []
    for i in range(T, T + len(val_set)):
        X_val.append(inputs[i-T:i, 0])
    X_val = np.array(X_val)
    y_val = sc.transform(val_set)

    # reshape to 3D array (format needed by LSTMs -> number of samples,timesteps, input dimension)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    return X_train, y_train, X_val, y_val

In [4]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

def LSTM(hidden_size, X_train):
    
    x = X_train.shape[1]
    model = Sequential()
    model.add(LSTM(units = hidden_size, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = hidden_size, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = hidden_size, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = hidden_size, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
    return model

In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics

#initialize the model paramters
batch_size = 16
hidden_size = 20
LR = 0.001
epochs = 100

#load the training and test data
X_train, y_train, X_val, y_val = data_loader()
#generate the model and compile it
model = LSTM(hidden_size, X_train)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
#train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data = (X_val, y_val), verbose=2)
predicted_stock_price = model.predict(X_val)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

TypeError: LSTM() got an unexpected keyword argument 'units'

In [5]:
import matplotlib.pyplot as plt

def plotter():
   # fig = plt.plot(real_stock_price, color = 'black', label = 'TATA Stock Price')
    fig = plt.plot(predicted_stock_price, color = 'green', label = 'Predicted TATA Stock Price')
    fig = plt.title('TATA Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('TATA Stock Price')
    plt.legend()
    plt.show()
    return fig